In [53]:
import numpy as np
import tensorflow as tf

def duplicate_each_element(T, num):  # duplicate each element 'num' times for last dimension
        temp = tf.expand_dims(T, -1)
        shape_list = [1] * len(T.get_shape()) + [num]
        temp = tf.tile(temp, shape_list)
        T_shape = T.get_shape().as_list()

        temp = tf.reshape(temp, [-1] + T_shape[1:-1] + [T_shape[-1] * num])
        return temp

def merge_last_two_dimensions(T):  ## T : (None, ....)
    T_shape = T.get_shape().as_list()
    new_shape = [-1] + T_shape[1:-2] + [T_shape[-2] * T_shape[-1]]
    temp = tf.reshape(T, new_shape)
    return temp

def split_last_dimension(T, last_size): ## T : (None, ... (n * last_size) )
    T_shape = T.get_shape().as_list()
    assert T_shape[-1] % last_size == 0
    new_shape = [-1] + T_shape[1:-1] + [T_shape[-1]/last_size] + [last_size]
    temp = tf.reshape(T, new_shape) ## output : (None, ..., n, last_size)
    return temp

def calc_iou(boxes1, boxes2):
    """calculate ious
    Args:
      boxes1: 5-D tensor [BATCH_SIZE, CELL_SIZE, CELL_SIZE, BOXES_PER_CELL, 4]  ====> (x_center, y_center, w, h)
      boxes2: 5-D tensor [BATCH_SIZE, CELL_SIZE, CELL_SIZE, BOXES_PER_CELL, 4] ===> (x_center, y_center, w, h)
    Return:
      iou: 4-D tensor [BATCH_SIZE, CELL_SIZE, CELL_SIZE, BOXES_PER_CELL]
    """

    # transform (x_center, y_center, w, h) to (x1, y1, x2, y2)
    boxes1_t = tf.stack([boxes1[..., 0] - boxes1[..., 2] / 2.0,
                         boxes1[..., 1] - boxes1[..., 3] / 2.0,
                         boxes1[..., 0] + boxes1[..., 2] / 2.0,
                         boxes1[..., 1] + boxes1[..., 3] / 2.0],
                        axis=-1)

    boxes2_t = tf.stack([boxes2[..., 0] - boxes2[..., 2] / 2.0,
                         boxes2[..., 1] - boxes2[..., 3] / 2.0,
                         boxes2[..., 0] + boxes2[..., 2] / 2.0,
                         boxes2[..., 1] + boxes2[..., 3] / 2.0],
                        axis=-1)

    # calculate the left up point & right down point
    lu = tf.maximum(boxes1_t[..., :2], boxes2_t[..., :2])
    rd = tf.minimum(boxes1_t[..., 2:], boxes2_t[..., 2:])

    # intersection
    intersection = tf.maximum(0.0, rd - lu)
    inter_square = intersection[..., 0] * intersection[..., 1]

    # calculate the boxs1 square and boxs2 square
    square1 = boxes1[..., 2] * boxes1[..., 3]
    square2 = boxes2[..., 2] * boxes2[..., 3]

    union_square = tf.maximum(square1 + square2 - inter_square, 1e-10)

    return tf.clip_by_value(inter_square / union_square, 0.0, 1.0)


# output = tf.reshape(tf.cast(tf.cast(tf.range(2 * 3 * 3 * 41), tf.float32)/100, tf.float32), [2, 3, 3, 41])
# output_gt = tf.reshape(tf.cast(tf.range(2 * 3 * 3 * 41), tf.float32), [2, 3, 3, 41])

# boxes_pred = tf.nn.sigmoid(output[..., :8])
# conf_pred = tf.nn.sigmoid(output[..., 8:12])
# cls_pred = tf.nn.softmax(output[..., -1 * 31:])
                                                                    

a = tf.constant([[1, 0, 1, 2], [0, 1, 0, 1], [1, 3, 0, 0]])

b = tf.expand_dims(a, -1)
c = duplicate_each_element(b, 4)
# b = tf.reshape(tf.tile(a, [1, 4]), [12, 4])
# c = tf.tile(a, [4, 1])
sess = tf.Session()
print sess.run(a)
# print sess.run(b)
print sess.run(c)




[[1 0 1 2]
 [0 1 0 1]
 [1 3 0 0]]
[[[1 1 1 1]
  [0 0 0 0]
  [1 1 1 1]
  [2 2 2 2]]

 [[0 0 0 0]
  [1 1 1 1]
  [0 0 0 0]
  [1 1 1 1]]

 [[1 1 1 1]
  [3 3 3 3]
  [0 0 0 0]
  [0 0 0 0]]]
